<b>Percolation</b>

In [1]:
# Cellule d'installation des packeges nécessaires (il faut au préalable installer graphviz sur Windows)

import string
import sys

# Install conda packages in the current Jupyter kernel
!conda update --yes -n base -c defaults conda

try:
    from tkinter import *
except:
    !conda install --yes --prefix {sys.prefix} tk
    from tkinter import *
    
try:
    from PIL import Image, ImageTk
except:
    !conda install --yes --prefix {sys.prefix} pillow
    from PIL import Image, ImageTk
    
try:
    import numpy as np
except:
    !conda install --yes --prefix {sys.prefix} numpy
    import numpy as np
    
try:
    import graphviz
except:
    !conda install --yes --prefix {sys.prefix} python-graphviz
    import graphviz

try:
    import pygraphviz
except:
    !conda install -c alubbock --yes --prefix {sys.prefix} graphviz pygraphviz
    import pygraphviz

try:
    import networkx as nx
except:
    !conda install --yes --prefix {sys.prefix} networkx
    import networkx as nx

#On utilise dot -Kneato avec pygraphviz (neato), mais il faut qu'il soit lancé avec tous les droits:
!dot -c

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
# Classe permettant la construction du graphe avec graphviz/pygraphviz

class Graph():
    def __init__(self, distances, labels, image_path="./out.png"):
        dt = [('len', float)]
        distances = distances.view(dt)
        
        self.G = nx.from_numpy_matrix(distances)
        
        positives = np.where(np.array(labels)==1)[0]
        color = ["red"]*len(positives)
        positives_attr = dict(zip(positives, color))
        nx.set_node_attributes(self.G, positives_attr, 'color')
        
        negatives = np.where(np.array(labels)==0)[0]
        color = ["green"]*len(negatives)
        negatives_attr = dict(zip(negatives, color))
        nx.set_node_attributes(self.G, negatives_attr, 'color')
            
        self.G = nx.relabel_nodes(self.G, dict(zip(range(len(self.G.nodes())),string.ascii_uppercase)))
        
        self.G = nx.drawing.nx_agraph.to_agraph(self.G)
        self.G.node_attr.update(style="filled")
        self.G.edge_attr.update(width="2.0")
        self.G.draw(image_path, format='png', prog='dot', args='-Kneato') #neato, dot, twopi, circo, fdp, nop, wc, acyclic, gvpr, gvcolor, ccomps, sccmap, tred, sfdp, unflatten

### TEST
"""
A = np.array([(0, 0.3, 0.4, 0.7),
               (0.3, 0, 0.9, 0.2),
               (0.4, 0.9, 0, 0.1),
               (0.7, 0.2, 0.1, 0)
               ])*10

l = len(A)*[0]
l[1] = 1

graph = Graph(A, l)
"""

'\nA = np.array([(0, 0.3, 0.4, 0.7),\n               (0.3, 0, 0.9, 0.2),\n               (0.4, 0.9, 0, 0.1),\n               (0.7, 0.2, 0.1, 0)\n               ])*10\n\nl = len(A)*[0]\nl[1] = 1\n\ngraph = Graph(A, l)\n'

In [3]:
# Classe permettant de mettre à jour le graphe au fur et à mesure de la percolation

class Percolation():
    def __init__(self, n=4, img="./out.png", safety_distance=2):
        self.x = np.zeros([n, n])
        indices = np.triu_indices(n, 1)
        distances = list(np.random.randint(1, 10, size=len(indices[0])))
        self.x[indices] = distances
        self.x += self.x.T
        self.safety_distance = safety_distance
        self.states = n*[0]
        self.img = img
        graph = Graph(self.x, self.states, self.img)
    
    def update(self):
        if (np.array(self.states) <= 0).all():
            patient_zero = np.random.randint(0, len(self.states))
            self.states[patient_zero] = 1
        else:
            positive_patients = list(np.where(np.array(self.states)==1)[0])
            
            for positive_patient in positive_patients:
                too_close_people =  np.where(self.x <= self.safety_distance)
                patients_contacts = too_close_people[1][np.where(too_close_people[0] == positive_patient)[0]]
                
                if len(patients_contacts)>0:
                
                    self.states = np.array(self.states)
                    self.states[patients_contacts] = 1
                    self.states = list(self.states)
        graph = Graph(self.x, self.states, self.img)

### TEST
"""
percolation = Percolation()
percolation.update()
percolation.update()
percolation.update()
"""

'\npercolation = Percolation()\npercolation.update()\npercolation.update()\npercolation.update()\n'

In [4]:
# Classe permettant d'afficher les résultats avec Tkinter

class Interface(Frame):
    def __init__(self, master=None, image_path="./out.png", nodes=4, safety_distance=2):
        Frame.__init__(self, master)
        self.master = master
        self.pack(fill=BOTH, expand=1)
        self.image_path = image_path
        self.percolation = Percolation(nodes, image_path, safety_distance)
        img = ImageTk.PhotoImage(Image.open(self.image_path))
        self.panel = Label(self.master, image=img)
        self.panel.pack(side="bottom", fill="both", expand="yes")
        
    def updater(self):
        self.percolation.update()
        img = ImageTk.PhotoImage(Image.open(self.image_path))
        self.panel.configure(image=img)
        self.panel.image = img
        self.after(1000, self.updater)

### TEST
"""
root = Tk()
app = Interface(master=root)
root.wm_title("Percolation interface")
root.geometry("1000x1000")

app.updater()

root.mainloop()
"""

'\nroot = Toplevel()\napp = Interface(master=root)\nroot.wm_title("Percolation interface")\nroot.geometry("1000x1000")\n\napp.updater()\n\nroot.mainloop()\n'

In [5]:
# Test avec n personnes et une distanciation de 2 sur 10

for i in range(3):
    out_path = "./out.png"
    n = 10
    safe_distance = 2

    root = Tk()
    app = Interface(master=root, image_path=out_path, nodes=n, safety_distance=safe_distance)
    root.wm_title("Percolation interface")
    root.geometry("1000x1000")

    app.updater()

    root.mainloop()